<a href="https://colab.research.google.com/github/serinamarie/DS-Unit-2-Regression-Classification/blob/master/Grill_Regression_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module4')

     |████████████████████████████████| 92kB 8.9MB/s 
     |████████████████████████████████| 133kB 32.3MB/s 
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.1)
     |████████████████████████████████| 614kB 32.3MB/s 
     |████████████████████████████████| 225kB 41.6MB/s 
     |████████████████████████████████| 768kB 48.0MB/s 
     |████████████████████████████████| 51kB 13.2MB/s 
     |████████████████████████████████| 194kB 51.4MB/s 
     |████████████████████████████████| 737kB 55.3MB/s 
     |████████████████████████████████| 51kB 24.7MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145035 sha256=a68cc7d733fbeb148b9dfa816816cdb253359f9cac49b8e957d0bdd826fd9442
  Stored in directory: /root/.cache/pip/wheels/ce/c7/f1/dbfef4848ebb048cb1d4a22d1ed0c62d8ff2523747235e19fe
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=750e4aa32815f8e7d793dcdb19158d6c46dc6ec2

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd

train_features = pd.read_csv('../data/tanzania/train_features.csv')
train_labels = pd.read_csv('../data/tanzania/train_labels.csv')
test_features = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

In [6]:
from sklearn.model_selection import train_test_split
X_train = train_features
y_train = train_labels['status_group']

X_train.shape, y_train.shape

((59400, 40), (59400,))

In [7]:
X_train.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625


In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, train_size = 0.80, test_size = 0.20,
    stratify = y_train, random_state=42
)
X_train.shape, X_val.shape, y_train.shape, y_val.shape


((47520, 40), (11880, 40), (47520,), (11880,))

In [0]:
import numpy as np
from scipy import stats
X_train_numeric = X_train.select_dtypes('number')

In [0]:
#X_train_numeric = X_train_numeric[X_train_numeric['construction_year'] > 0]
#X_train_numeric = X_train_numeric[X_train_numeric['population'] > 1]
#X_train_numeric = X_train_numeric.loc[X_train_numeric['construction_year'] > 1].dropna()
#X_train = X_train_numeric.merge(train_features, how='left')
#y_train = X_train.merge(train_labels, on='id', how='inner').drop(columns=['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private', 'region_code', 'district_code', 'population','construction_year','date_recorded', 'funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by', 'scheme_management', 'scheme_name', 'permit', 'extraction_type', 'extraction_type_group', 'extraction_type_class', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group'])
#X_train.head()

In [13]:
import category_encoders as ce
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#Pinpoint features
categorical_features = ['quantity_group', 'quantity', 'source_class', 'payment', 'source_type', 'management', 'extraction_type', 'quality_group', 'waterpoint_type_group', 'basin', 'region_code', 'lga', 'public_meeting']
numeric_features = ['amount_tsh', 'gps_height', 'longitude', 'latitude','construction_year', 'population', 'num_private']

features = categorical_features + numeric_features

#Make a subset of data with our specified features
X_train_subset = X_train[features]
X_val_subset = X_val[features]

#encode it
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)

#scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_val_scaled = scaler.transform(X_val_encoded)

from sklearn.ensemble import RandomForestClassifier
m = RandomForestClassifier(n_estimators=200,min_samples_leaf=2 ,n_jobs=-1,max_features=0.25)
m.fit(X_train_scaled, y_train)
y_pred= m.predict(X_val_scaled)
accuracy_score(y_val, y_pred)

0.8154882154882155

In [0]:
X_test_subset = test_features[features]
X_test_encoded = encoder.transform(X_test_subset)
X_test_scaled = scaler.transform(X_test_encoded)
assert all(X_test_encoded.columns == X_train_encoded.columns)

In [0]:
y_pred = m.predict(X_test_scaled)

In [0]:
#submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-03.csv',index=False)

In [0]:
#!head submission-03.csv

In [0]:
#if in_colab:
    from google.colab import files
    files.download('submission-03.csv')